# 1. load libraries

In [ ]:
library(behavr)
library(scopr)
library(sleepr)
library(tidyverse)
library(ggetho)
library(pheatmap)
library(grid)
library(ggfortify)
library(ggrepel)

# 2. prepare metadata

In [ ]:
# there are different datasets, for calculating sleep parameters I want: 1. LDLD(2) first two days, 2. SD (motor=Rotated) first 36h, 3. from datasets SD_6 and SD_7: Control sleep

metadata_combined <- fread("/Users/Joana/sleep_signature/behaviour/DGRP/metadata_DGRP_combined.csv")

metadata$selection=ifelse(metadata$DGRP %in% c("88","287","303", "313", "359", "379", "441", "646", "892", "908"),"selected","dismissed")

metadata_SD <- metadata_combined[motor == "Rotated"]

metadata_combined$replicate <- 1:length(metadata_combined$DGRP)
metadata_SD$replicate <- 1:length(metadata_SD$DGRP)

# curation (no sleep in one recorded night)
metadata_exclude <- metadata_combined[!(replicate %in% c('83', '699', '63', '118', '764', '150', '151', '152', '602', '427', '312', '10', '661', '599', '379', '11', '663'))]

metadata_SD <- metadata_SD[!(replicate %in% c("211", "282", "283", "284", "285", "286", "287", "217"))]
metadata_sleep_exclude <- metadata_exclude[motor == "Control" & dataset %in% c("SD_6", "SD_7")]
metadata_LDLD_exclude <- metadata_exclude[dataset %in% c("LDLD", "LDLD2")]

# 3. linking different metadata files to results files

In [ ]:
metadata_LDLD_exclude <- link_ethoscope_metadata(metadata_LDLD_exclude, 
                                      result_dir = "/2021-04-14_ethoscope_data/results")

system.time(
  dt <- load_ethoscope(metadata,
                          reference_hour=NA, 
                          FUN = sleepr::sleep_annotation,
                          # FUN_filter = scopr::Mode_filter,
                          cache = "ethoscope_cache", 
                          map_arg=list(velocity_correction_coef="velocity_correction_coefficient"),
                          verbose=FALSE)
)


# 4. adding light and dark phase column

In [ ]:
dt_sleep_exclude[, phase := ifelse(t %% hours(24) < hours(12), "L", "D")]
dt_LDLD_exclude[, phase := ifelse(t %% hours(24) < hours(12), "L", "D")]
dt_SD[, phase := ifelse(t %% hours(24) < hours(12), "L", "D")]

# 5. combine different metadata and dt files

In [ ]:
# make combined dt and metadata for all control sleep flies (Control from metadata_DGRP_combined) and the first 36h of the SD flies (Rotated from metadata_DGRP_combined) + first 48h of LDLD(2) 

dt_first_36h  <- dt_SD[t %between%  c(days(0), days(1) + hours(12))]
dt_LDLD_48h <- dt_LDLD_exclude[t %between%  c(days(0), days(2))]

dt <- rbindlist(list(dt_sleep_exclude, dt_first_36h, dt_LDLD_48h))
metadata <- rbindlist(list(metadata_sleep_exclude, metadata_SD, metadata_LDLD_exclude))

#redefine key for both dt and metadata to be id
setkey(metadata, 'id')
setkey(dt, 'id')

#combine dt and metadata
setmeta(dt, metadata)
print(dt)

# 6. make population plots

In [ ]:
ggetho(dt, aes(y=asleep, color=selection)) +
  stat_pop_etho() +
  stat_ld_annotations()+
  #facet_grid(DGRP ~ .)+
  #facet_grid(replicate ~ .)+
  theme_classic(base_size=30)+
  #theme(axis.title.x=element_blank(),axis.text.x=element_blank(),axis.ticks.x=element_blank())+
  scale_y_continuous("Time asleep", limits=c(0,1), labels = scales::percent) +
  scale_colour_manual(name=element_blank(), values = c('#49007e', '#e99500', 'black')) +
  scale_fill_manual(name=element_blank(), values = c('#49007e', '#e99500', 'black'))
# scale_colour_manual(values = c("black", "green", "magenta", "cyan"))
# coord_cartesian(xlim=c(days(1.5), days(3)))
#theme(axis.text.y=element_blank(),axis.ticks.y=element_blank())+

# 7. calculate and plot sleep duration by id

In [ ]:
# make new dt with calculations of mean during L and D
summary_dt <- 
  rejoin(dt[,
                    .(
                      # sleep_fraction_all = mean(asleep),
                      sleep_fraction_l = mean(asleep[phase == "L"]),
                      sleep_fraction_d = mean(asleep[phase == "D"])
                    ),
                    ,by=id])
summary_dt

# make summary_dt_melted to make two variables: 'phase' and actual value in 'sleep_fraction'
summary_dt_melted <- melt(summary_dt, measure.vars = patterns("sleep_fraction_"),
                          variable.name = "phase", value.name = "sleep_fraction")

## plotting sleep fraction during night time, day time or total sleep
ggplot(summary_dt_melted, aes(x=factor(DGRP), y=sleep_fraction, colour=phase)) + 
  geom_boxplot(outlier.colour = NA, varwidth = TRUE) +
  # geom_point()+
  # geom_jitter(alpha=.7) +
  # facet_grid(DGRP_replicate ~ .) +
  scale_y_continuous(name= "Fraction of time asleep",labels = scales::percent)+
  # scale_x_discrete(name= "sleep quality")+
  # scale_colour_manual(name= "phase", values = c("darkgrey", "black"), labels = c("light", "dark"))+
  theme_bw(base_size = 15)
##geom_jitter(aes(shape=phase))
## make subset of a few lines from A and D then make the above boxplot with shape and colour

#calculate significance ANOVA
res.aov_sleepfraction_d <- aov(sleep_fraction_d ~ selection, data = summary_dt)
summary(res.aov_sleepfraction_d)
TukeyHSD(res.aov_sleepfraction_d)

# 8. calculate sleep bouts and latency

In [ ]:
# make overall_summary including all calculations (sleep durations, bout length and number, sleep latencies)
bout_dt <- bout_analysis(asleep, dt)
bout_dt  <- bout_dt [asleep == TRUE, -"asleep"]

bout_dt_dark  <- bout_dt [t %between%  c(days(0)+ hours(12), days(1)) | t %between% c(days(1)+ hours(12), days(2)) | t %between% c(days(2)+ hours(12), days(3))]
bout_summary_0  <- bout_dt_dark [,
                                         .(n_bouts = .N,
                                           mean_bout_length = mean(duration)),
                                         by=id]

bout_dt_dark <- bout_dt[t %between%  c(days(0) + hours(12), days(1))]
# We express t relatively to the first day
bout_dt_dark[, t:= t - days(0)]

bout_summary_1 <- bout_dt_dark[,.(
  latency_1 = t[1], # the first bout is at t[1] 
  first_bout_length_1 = duration[1],
  latency_to_longest_bout_1 = t[which.max(duration)],
  length_longest_bout_1 = max(duration)
),
by=id]

bout_dt_dark <- bout_dt[t %between%  c(days(1) + hours(12), days(2))]
# We express t relatively to the first day
bout_dt_dark[, t:= t - days(1)]

bout_summary_2 <- bout_dt_dark[,.(
  latency_2 = t[1], # the first bout is at t[1] 
  first_bout_length_2 = duration[1],
  latency_to_longest_bout_2 = t[which.max(duration)],
  length_longest_bout_2 = max(duration)
),
by=id]

bout_dt_dark <- bout_dt[t %between%  c(days(2) + hours(12), days(3))]
# We express t relatively to the first day
bout_dt_dark[, t:= t - days(2)]

bout_summary_3 <- bout_dt_dark[,.(
  latency_3 = t[1], # the first bout is at t[1] 
  first_bout_length_3 = duration[1],
  latency_to_longest_bout_3 = t[which.max(duration)],
  length_longest_bout_3 = max(duration)
),
by=id]

bout_summaries <- list(bout_summary_0, bout_summary_1, bout_summary_2, bout_summary_3)
bout_summary <- Reduce(function(x, y) merge(x, y, all=TRUE), bout_summaries) 
bout_summary

#average of the latencies of first and second night for each animal, there is no 2nd night for the cut SD flies (dt_first_36h), so we need enter simply the value for the latency_1 for these flies
bout_summary$latency <- rowMeans(bout_summary[,c('latency_1', 'latency_2', 'latency_3')], na.rm = TRUE)
bout_summary$latency_to_longest_bout <- rowMeans(bout_summary[,c('latency_to_longest_bout_1', 'latency_to_longest_bout_2', 'latency_to_longest_bout_3')], na.rm = TRUE)

# rejoin summary_dt(all metadata variables, sleep_fraction_all, sleep_fraction_l, sleep_fraction_d) and bout_summary (n_bouts, mean_bout_length, latency)
overall_summary  <- summary_dt [bout_summary]
overall_summary

# 9. summarize sleep parameters in heatmap

In [ ]:
heatmap_dt <- overall_summary[,.(
  selection = max(selection),
  sleep_fraction_day = mean(sleep_fraction_l),
  sleep_fraction_day_std = sd(sleep_fraction_l),
  sleep_fraction_night = mean(sleep_fraction_d),
  sleep_fraction_night_std = sd(sleep_fraction_d),
  n_bouts = mean(n_bouts),
  n_bouts_std = sd(n_bouts),
  mean_bout_length = mean(mean_bout_length),
  mean_bout_length_std = sd(mean_bout_length),
  latency = mean(latency),
  latency_std = sd(latency),
  latency_to_longest_bout = mean(latency_to_longest_bout),
  latency_to_longest_bout_std = sd(latency_to_longest_bout)
),
by=DGRP]

heatmap_z <- heatmap_dt %>% 
  mutate(sleep_fraction_day = (sleep_fraction_day - mean(sleep_fraction_day))/sd(sleep_fraction_day),
         sleep_fraction_day_std = (sleep_fraction_day_std - mean(sleep_fraction_day_std))/sd(sleep_fraction_day_std),
         sleep_fraction_night = (sleep_fraction_night - mean(sleep_fraction_night))/sd(sleep_fraction_night),
         sleep_fraction_night_std = (sleep_fraction_night_std - mean(sleep_fraction_night_std))/sd(sleep_fraction_night_std),
         n_bouts = (n_bouts - mean(n_bouts))/sd(n_bouts),
         n_bouts_std = (n_bouts_std - mean(n_bouts_std))/sd(n_bouts_std),
         mean_bout_length = (mean_bout_length - mean(mean_bout_length))/sd(mean_bout_length),
         mean_bout_length_std = (mean_bout_length_std - mean(mean_bout_length_std))/sd(mean_bout_length_std),
         latency = (latency - mean(latency))/sd(latency),
         latency_std = (latency_std - mean(latency_std))/sd(latency_std),
         latency_to_longest_bout = (latency_to_longest_bout - mean(latency_to_longest_bout))/sd(latency_to_longest_bout),
         latency_to_longest_bout_std = (latency_to_longest_bout_std - mean(latency_to_longest_bout_std))/sd(latency_to_longest_bout_std))

library("writexl")
write_xlsx(heatmap_z,"/Users/Joana/sleep_signature/behaviour/DGRP/heatmap_z_04072022.xlsx")

heatmap_z$colors=ifelse(heatmap_z$DGRP %in% c("88","287","303", "313", "359", "379", "441", "646", "892", "908"),"selected","dismissed")

heatmap_z <- heatmap_z %>%
  remove_rownames() %>%
  column_to_rownames(var = 'DGRP')

heatmap=pheatmap(heatmap_z[, 1:12], 
                 clustering_distance_rows="correlation",
                 clustering_method="ward.D")
cols=heatmap_z[order(match(rownames(heatmap_z), heatmap$gtable$grobs[[5]]$label)), ]$colors
heatmap$gtable$grobs[[5]]$gp=gpar(col=cols)
heatmap

# 10. calculate PCA and plot

In [ ]:
df <- heatmap_z[,4:15]
pca_res <- prcomp(df, scale. = TRUE)

autoplot(pca_res)

autoplot(pca_res, data = heatmap_z, colour = "selection", label=TRUE, shape="selection", frame=T, label.size=5) +
  theme_classic() +
  scale_colour_manual(name=element_blank(), values = c('#49007e', '#dd6d00')) +
  scale_fill_manual(name=element_blank(), values = c('#49007e', '#dd6d00'))

# 11. calculate pDoze and pWake and plot

In [ ]:
source('transition_functions.R')
dt_pwake<-behavr::bin_apply_all(data = dt, y=asleep, FUN=p_wake)
dt_amount<-behavr::bin_apply_all(data = dt, y=asleep, FUN=mean)

dt_pwake$p_wake <- dt_pwake$asleep
dt_pwake$asleep <- NULL
dt_pwake <- merge(dt_amount, dt_pwake, by=c('id', 't'))

dt_pdoze<-behavr::bin_apply_all(data = dt, y=asleep, FUN=p_doze)
dt_amount<-behavr::bin_apply_all(data = dt, y=asleep, FUN=mean)

dt_pdoze$p_doze <- dt_pdoze$asleep
dt_pdoze$asleep <- NULL

dt_transitions <- merge(dt_pwake, dt_pdoze, by=c('id', 't'))

#NIGHT
dt_transitions_night  <- dt_transitions [t %between%  c(days(0)+ hours(12), days(1)) | t %between% c(days(1)+ hours(12), days(2)) | t %between% c(days(2)+ hours(12), days(3))]
dt_transitions_id  <- dt_transitions_night [,
                                   .(mean_pwake = mean(p_wake, na.rm = TRUE),
                                   mean_pdoze = mean(p_doze, na.rm = TRUE)),
                                 by=id]

summary  <- overall_summary [dt_transitions_id]

dt_transitions_DGRP  <- summary [,
                                 .(mean_pwake = mean(mean_pwake),
                                   mean_pdoze = mean(mean_pdoze)),
                                 by=DGRP]

dt_transitions_DGRP <- dt_transitions_DGRP[!dt_transitions_DGRP$DGRP == "NA"]
dt_transitions_DGRP$selection=ifelse(dt_transitions_DGRP$DGRP %in% c("88","287","303", "313", "359", "379", "441", "646", "892", "908"),"selected","dismissed")
dt_transitions_DGRP$colors=ifelse(dt_transitions_DGRP$DGRP %in% c("88","287","303", "313", "359", "379", "441", "646", "892", "908"),"#e99500","#49007e")

#DAY
dt_transitions_day  <- dt_transitions [t %between%  c(days(0), days(0) + hours(12)) | t %between% c(days(1), days(1) + hours(12)) | t %between% c(days(2), days(2) + hours(12))]

dt_transitions_id  <- dt_transitions_day [,
                                            .(mean_pwake = mean(p_wake, na.rm = TRUE),
                                              mean_pdoze = mean(p_doze, na.rm = TRUE)),
                                            by=id]
summary  <- overall_summary [dt_transitions_id]
dt_transitions_DGRP  <- summary [,
                                 .(mean_pwake = mean(mean_pwake),
                                   mean_pdoze = mean(mean_pdoze)),
                                 by=DGRP]

dt_transitions_DGRP <- dt_transitions_DGRP[!dt_transitions_DGRP$DGRP == "NA"]
dt_transitions_DGRP$selection=ifelse(dt_transitions_DGRP$DGRP %in% c("88","287","303", "313", "359", "379", "441", "646", "892", "908"),"selected","dismissed")
dt_transitions_DGRP$colors=ifelse(dt_transitions_DGRP$DGRP %in% c("88","287","303", "313", "359", "379", "441", "646", "892", "908"),"#e99500","#49007e")

#DAY AND NIGHT
dt_transitions_id_day_night  <- dt_transitions [,
                                        .(mean_pwake = mean(p_wake, na.rm = TRUE),
                                          mean_pdoze = mean(p_doze, na.rm = TRUE)),
                                        by=id]

summary  <- overall_summary [dt_transitions_id_day_night]

dt_transitions_DGRP_day_night  <- summary [,
                                 .(mean_pwake = mean(mean_pwake),
                                   mean_pdoze = mean(mean_pdoze)),
                                 by=DGRP]

dt_transitions_DGRP_day_night$selection=ifelse(dt_transitions_DGRP_day_night$DGRP %in% c("88","287","303", "313", "359", "379", "441", "646", "892", "908"),"selected","dismissed")

p <- ggplot(dt_transitions_DGRP, aes(x=mean_pwake, y=mean_pdoze, colour=selection)) + geom_point()
p + stat_ellipse(type = "norm") +
  theme_classic(base_size = 20) +
  scale_colour_manual(name=element_blank(), values = c('#49007e', '#e99500', 'black')) +
  scale_fill_manual(name=element_blank(), values = c('#49007e', '#e99500', 'black')) +
  geom_text_repel(aes(label=dt_transitions_DGRP$DGRP), size=8, max.overlaps = 10, color=dt_transitions_DGRP$colors)
  #scale_x_continuous(limits = c(0, 0.5)) + 
  #scale_y_continuous(limits = c(0, 0.5))